### Create the mart layer of the air travel lakehouse

##### Declare the global variables

In [ ]:
project_id = "cs378-fa2025"
region = "us-central1"
dataset = "air_travel_mrt"
region = "us-central1"

In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client()

dataset_id = bigquery.Dataset(f"{project_id}.{dataset}")
dataset_id.location = region
resp = bq_client.create_dataset(dataset_id, exists_ok=True)
print("Created dataset {}.{}".format(bq_client.project, resp.dataset_id))

Created dataset cs378-fa2025.air_travel_mrt


### **User journey 1**: I oversee the business development strategy for Starbucks. I want to understand airport traffic patterns so that I can figure out how to optimize Starbuck's presence at major US airports.

##### Query strategy: join the tsa traffic data (TSA_Traffic) with the airport maps data (Airport_Concessions, Airport_Establishment) via the airport data (Airport).

##### Query 1: Get the average monthly traffic for Starbucks per airport across all years. The foot traffic is based on the TSA data which is linked to a security checkpoint, but not a terminal.

In [ ]:
%%bigquery
select extract(month from t.event_date) as month_of_year, t.airport_icao, a.name as airport_name,
  ac.location, round(avg(total_traffic), 2) avg_foot_traffic
from air_travel_int.TSA_Traffic t
join air_travel_int.Airport a on t.airport_icao = a.icao
join air_travel_int.Airport_Concessions ac on a.icao = ac.airport_icao
where ac.business = 'Starbucks'
group by extract(month from t.event_date), t.airport_icao, a.name, ac.location
order by airport_icao, month_of_year

Query is running:   0%|          |

Downloading:   0%|          |

,month_of_year,airport_icao,airport_name,location,avg_foot_traffic
0,2,KAUS,Austin Bergstrom International Airport,Near Gate 14,422.20
1,11,KAUS,Austin Bergstrom International Airport,Near Gate 14,177.33
2,2,KBDL,Bradley International Airport,"Food court area, post-security",178.67
3,11,KBDL,Bradley International Airport,"Food court area, post-security",1.00
4,2,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near gate A2",169.00
5,2,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near Food Court and entrance to C...",169.00
6,11,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near gate A2",147.00
7,11,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near Food Court and entrance to C...",147.00
8,2,KFLL,Fort Lauderdale Hollywood International Airport,Baggage Claim Level,455.80
9,2,KFLL,Fort Lauderdale Hollywood International Airport,"Departure Level, near Gate C4",455.80


##### Query 2: Same query as before, but with the month of the year translated from its numeric value to string

In [ ]:
%%bigquery
select case month_of_year
  when 1 then 'January'
  when 2 then 'February'
  when 3 then 'March'
  when 4 then 'April'
  when 5 then 'May'
  when 6 then 'June'
  when 7 then 'July'
  when 8 then 'August'
  when 9 then 'September'
  when 10 then 'October'
  when 11 then 'November'
  when 12 then 'December' end as month_long, * except (month_of_year)
from
(select extract(month from t.event_date) as month_of_year, t.airport_icao, a.name as airport_name,
  ac.location, round(avg(total_traffic), 2) avg_foot_traffic
from air_travel_int.TSA_Traffic t
join air_travel_int.Airport a on t.airport_icao = a.icao
join air_travel_int.Airport_Concessions ac on a.icao = ac.airport_icao
where ac.business = 'Starbucks'
group by extract(month from t.event_date), t.airport_icao, a.name, ac.location
order by airport_icao, month_of_year)

Query is running:   0%|          |

Downloading:   0%|          |

,month_long,airport_icao,airport_name,location,avg_foot_traffic
0,February,KAUS,Austin Bergstrom International Airport,Near Gate 14,422.20
1,November,KAUS,Austin Bergstrom International Airport,Near Gate 14,177.33
2,February,KBDL,Bradley International Airport,"Food court area, post-security",178.67
3,November,KBDL,Bradley International Airport,"Food court area, post-security",1.00
4,February,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near gate A2",169.00
5,February,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near Food Court and entrance to C...",169.00
6,November,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near gate A2",147.00
7,November,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near Food Court and entrance to C...",147.00
8,February,KFLL,Fort Lauderdale Hollywood International Airport,Baggage Claim Level,455.80
9,February,KFLL,Fort Lauderdale Hollywood International Airport,"Departure Level, near Gate C4",455.80


##### Materialize the query results into a data mart

In [ ]:
%%bigquery
create or replace table air_travel_mrt.foot_traffic_starbucks_airports_monthly as
  select case month_of_year
  when 1 then 'January'
  when 2 then 'February'
  when 3 then 'March'
  when 4 then 'April'
  when 5 then 'May'
  when 6 then 'June'
  when 7 then 'July'
  when 8 then 'August'
  when 9 then 'September'
  when 10 then 'October'
  when 11 then 'November'
  when 12 then 'December' end as month_long, * except (month_of_year)
from
(select extract(month from t.event_date) as month_of_year, t.airport_icao, a.name as airport_name,
  ac.location, round(avg(total_traffic), 2) avg_foot_traffic
from air_travel_int.TSA_Traffic t
join air_travel_int.Airport a on t.airport_icao = a.icao
join air_travel_int.Airport_Concessions ac on a.icao = ac.airport_icao
where ac.business = 'Starbucks'
group by extract(month from t.event_date), t.airport_icao, a.name, ac.location
order by airport_icao, month_of_year)

Query is running:   0%|          |

""


##### Check the mart

In [ ]:
%%bigquery
select *
from air_travel_mrt.foot_traffic_starbucks_airports_monthly

Query is running:   0%|          |

Downloading:   0%|          |

,month_long,airport_icao,airport_name,location,avg_foot_traffic
0,February,KLAS,McCarran International Airport,Gate C,379.31
1,February,KLAX,Los Angeles International Airport,Near Gate 27,187.00
2,February,KBDL,Bradley International Airport,"Food court area, post-security",178.67
3,November,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near Food Court and entrance to C...",147.00
4,November,KLAS,McCarran International Airport,Gate C,11.50
5,February,KFLL,Fort Lauderdale Hollywood International Airport,Baggage Claim Level,455.80
6,November,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near gate A2",147.00
7,November,KSFO,San Francisco International Airport,Level 1 Arrivals,17.57
8,November,KLAX,Los Angeles International Airport,Near Gate 27,7.60
9,February,KSAV,Savannah Hilton Head International Airport,"Pre-security, near Ticket Counters",132.00


##### Query 3: Now use the LLM to look up the terminal of a security checkpoint. This will allow us to report more granular traffic data for an airport establishment. Store the security checkpoint to terminal mappings as a tmp table for now. Future work includes incorporating this step into the int layer.

In [ ]:
%%bigquery
create or replace table air_travel_tmp.tsa_checkpoint_terminal as
  with tsa_checkpoints as (
    select distinct security_checkpoint, airport_icao
    from air_travel_int.TSA_Traffic
  ),
  tsa_checkpoint_terminal as (
     select airport_icao,
      security_checkpoint,
      AI.GENERATE(
        ('What is the terminal for TSA security checkpoint ', security_checkpoint, ' and airport ', airport_icao, '? If you cannot find it, return None.'),
        connection_id => 'us-central1.vertex-ai-connection',
        endpoint => 'gemini-2.5-flash-lite',
        model_params => JSON '{"generation_config":{"thinking_config": {"thinking_budget": 0}}, "tools": [{"googleSearch": {}}]}',
        output_schema => 'terminal STRING') as llm_response
   from tsa_checkpoints
  )
select airport_icao, security_checkpoint, llm_response.terminal as mapped_terminal
from tsa_checkpoint_terminal

Query is running:   0%|          |

""


##### Check the results

In [ ]:
%%bigquery
select * from air_travel_tmp.tsa_checkpoint_terminal

Query is running:   0%|          |

Downloading:   0%|          |

,airport_icao,security_checkpoint,mapped_terminal
0,KABQ,Checkpoint,1
1,KLAS,Terminal 1 - E-Gates,1
2,KLAS,Terminal 1 - C/D,1
3,KPHF,PHF Checkpoint,1
4,PHNL,HNL05,1
...,...,...,...
327,KLAX,Terminal 7 - Passenger,Terminal B
328,KMCI,MCI-B Checkpoint,Terminal B
329,KMSY,Checkpoint 1,Terminal B
330,KMCO,West Checkpoint,Terminal C


##### Incorporate the mapping table into the earlier query to obtain more granular traffic data

In [ ]:
%%bigquery
select case month_of_year
  when 1 then 'January'
  when 2 then 'February'
  when 3 then 'March'
  when 4 then 'April'
  when 5 then 'May'
  when 6 then 'June'
  when 7 then 'July'
  when 8 then 'August'
  when 9 then 'September'
  when 10 then 'October'
  when 11 then 'November'
  when 12 then 'December' end as month_long, * except (month_of_year)
from
(select extract(month from t.event_date) as month_of_year, ct.mapped_terminal as terminal, t.airport_icao, a.name as airport,
  ac.location, round(avg(total_traffic), 2) avg_foot_traffic
from air_travel_int.TSA_Traffic t
join air_travel_tmp.tsa_checkpoint_terminal ct
  on t.airport_icao = ct.airport_icao and t.security_checkpoint = ct.security_checkpoint
join air_travel_int.Airport a on t.airport_icao = a.icao
join air_travel_int.Airport_Concessions ac on a.icao = ac.airport_icao
join air_travel_int.Airport_Concessions ac2
  on ac2.airport_icao = ct.airport_icao and ac2.terminal = ct.mapped_terminal
where ac.business = 'Starbucks'
group by extract(month from t.event_date), ct.mapped_terminal, t.airport_icao, a.name, ac.location
order by airport_icao, month_of_year)

Query is running:   0%|          |

Downloading:   0%|          |

,month_long,terminal,airport_icao,airport,location,avg_foot_traffic
0,February,Terminal 1,KFLL,Fort Lauderdale Hollywood International Airport,Baggage Claim Level,282.67
1,February,Terminal 1,KFLL,Fort Lauderdale Hollywood International Airport,"Departure Level, near Gate C4",282.67
2,February,Terminal 1,KFLL,Fort Lauderdale Hollywood International Airport,"Ticketing-Gates Level, near Gate D1",282.67
3,November,Terminal 1,KFLL,Fort Lauderdale Hollywood International Airport,Baggage Claim Level,142.00
4,November,Terminal 1,KFLL,Fort Lauderdale Hollywood International Airport,"Departure Level, near Gate C4",142.00
5,November,Terminal 1,KFLL,Fort Lauderdale Hollywood International Airport,"Ticketing-Gates Level, near Gate D1",142.00
6,February,6,KLAX,Los Angeles International Airport,Near Gate 27,127.50
7,February,6,KLAX,Los Angeles International Airport,"South Concourse, near Gate 150",127.50
8,November,6,KLAX,Los Angeles International Airport,Near Gate 27,5.00
9,November,6,KLAX,Los Angeles International Airport,"South Concourse, near Gate 150",5.00


##### Materialize the query results

In [ ]:
%%bigquery
create or replace table air_travel_mrt.foot_traffic_starbucks_terminals_monthly as
  select case month_of_year
  when 1 then 'January'
  when 2 then 'February'
  when 3 then 'March'
  when 4 then 'April'
  when 5 then 'May'
  when 6 then 'June'
  when 7 then 'July'
  when 8 then 'August'
  when 9 then 'September'
  when 10 then 'October'
  when 11 then 'November'
  when 12 then 'December' end as month_long, * except (month_of_year)
from
(select extract(month from t.event_date) as month_of_year, ct.mapped_terminal as terminal, t.airport_icao, a.name as airport,
  ac.location, round(avg(total_traffic), 2) avg_foot_traffic
from air_travel_int.TSA_Traffic t
join air_travel_tmp.tsa_checkpoint_terminal ct
  on t.airport_icao = ct.airport_icao and t.security_checkpoint = ct.security_checkpoint
join air_travel_int.Airport a on t.airport_icao = a.icao
join air_travel_int.Airport_Concessions ac on a.icao = ac.airport_icao
join air_travel_int.Airport_Concessions ac2
  on ac2.airport_icao = ct.airport_icao and ac2.terminal = ct.mapped_terminal
where ac.business = 'Starbucks'
group by extract(month from t.event_date), ct.mapped_terminal, t.airport_icao, a.name, ac.location
order by airport_icao, month_of_year)

Query is running:   0%|          |

""


##### Check the results

In [ ]:
%%bigquery
select * from air_travel_mrt.foot_traffic_starbucks_terminals_monthly

Query is running:   0%|          |

Downloading:   0%|          |

,month_long,terminal,airport_icao,airport,location,avg_foot_traffic
0,February,Terminal 1,KFLL,Fort Lauderdale Hollywood International Airport,Baggage Claim Level,282.67
1,February,Terminal 1,KFLL,Fort Lauderdale Hollywood International Airport,"Departure Level, near Gate C4",282.67
2,November,6,KLAX,Los Angeles International Airport,Near Gate 27,5.00
3,November,6,KLAX,Los Angeles International Airport,"South Concourse, near Gate 150",5.00
4,February,Terminal 1,KFLL,Fort Lauderdale Hollywood International Airport,"Ticketing-Gates Level, near Gate D1",282.67
5,February,6,KLAX,Los Angeles International Airport,Near Gate 27,127.50
6,February,Terminal 1,KSFO,San Francisco International Airport,Level 1 Arrivals,90.67
7,November,Terminal 1,KFLL,Fort Lauderdale Hollywood International Airport,Baggage Claim Level,142.00
8,November,Terminal 1,KFLL,Fort Lauderdale Hollywood International Airport,"Departure Level, near Gate C4",142.00
9,November,Terminal 1,KSFO,San Francisco International Airport,Level 1 Arrivals,37.67


##### Although we were able to create the second data mart, the terminal data is not clean and makes for a suboptional join key.

##### Query 4: Look at the data from a weekday perspective, over all months and years

In [ ]:
%%bigquery
select extract(dayofweek from t.event_date) as weekday, t.airport_icao, a.name as airport_name,
  ac.location, round(avg(total_traffic), 2) avg_foot_traffic
from air_travel_int.TSA_Traffic t
join air_travel_int.Airport a on t.airport_icao = a.icao
join air_travel_int.Airport_Concessions ac on a.icao = ac.airport_icao
where ac.business = 'Starbucks'
group by extract(dayofweek from t.event_date), t.airport_icao, a.name, ac.location
order by airport_icao, weekday

Query is running:   0%|          |

Downloading:   0%|          |

,weekday,airport_icao,airport_name,location,avg_foot_traffic
0,2,KAUS,Austin Bergstrom International Airport,Near Gate 14,330.38
1,2,KBDL,Bradley International Airport,"Food court area, post-security",134.25
2,2,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near Food Court and entrance to C...",161.67
3,2,KBHM,Birmingham-Shuttlesworth International Airport,"Upper Level, near gate A2",161.67
4,2,KFLL,Fort Lauderdale Hollywood International Airport,Baggage Claim Level,427.27
5,2,KFLL,Fort Lauderdale Hollywood International Airport,"Departure Level, near Gate C4",427.27
6,2,KFLL,Fort Lauderdale Hollywood International Airport,"Ticketing-Gates Level, near Gate D1",427.27
7,2,KLAS,McCarran International Airport,"Gate B, adjacent to the PGA Tour Grill",330.27
8,2,KLAS,McCarran International Airport,Gate C,330.27
9,2,KLAS,McCarran International Airport,Level 2 / Baggage Claim & Ticketing Level,330.27


In [ ]:
%%bigquery
select distinct extract(dayofweek from event_date) as weekday
from air_travel_int.TSA_Traffic

Query is running:   0%|          |

Downloading:   0%|          |

,weekday
0,2


##### We only have data for one day of the week (Tuesday), so there is no point in creating a weekday mart

##### Two areas of future work:

*  Remodel the security checkpoint data such that each checkpoint is labeled with a proper airport terminal that is joinable with the airport concessions so that we can estimate the foot traffic for a terminal
*  Go back and take another look at the batch prediction job to see why only a subset of the tsa traffic data loaded into the table. We should have had multiple week days of data in the table. We also should have had data for more months of the year



### **User journey 2**: I am a transportation researcher and I specialize in airports. I'm looking to understand the passenger experience at airports around the world. Here are the dimensions that I'm focused on:
*   Ease of finding your way (wayfinding)
*   Courtesy and helpfulness of staff
*   Cleanliness of washrooms and terminals
*   Wait times at check-in
*   Security and passport control
*   Quality of restaurants and shops






##### First, what categories (broad_category and sub_category) exist in the `Airport_Reviews` table?

In [ ]:
%%bigquery
select distinct broad_category, sub_category
from air_travel_int.Airport_Review
order by broad_category, sub_category

Query is running:   0%|          |

Downloading:   0%|          |

,broad_category,sub_category
0,Boarding & Departure,Airport Staff
1,Boarding & Departure,Baggage Claim
2,Boarding & Departure,Boarding Process
3,Boarding & Departure,Connecting Flights
4,Boarding & Departure,Customs
5,Boarding & Departure,Deplaning & Arrival
6,Boarding & Departure,Flight Operations
7,Boarding & Departure,Gate Area
8,Boarding & Departure,Miscellaneous
9,Boarding & Departure,Safety & Health


##### KPI to category mappings:  
*   Ease of finding your way: 'Wayfinding & Information'
*   Courtesy and helpfulness of staff: 'Airport Staff'
*   Cleanliness of washrooms and terminals: 'Amenities & Facilities'
*   Wait times at check-in: 'Boarding Process'
*   Security and passport control: 'Security Screening'
*   Quality of restaurants and shops: 'Food, Beverage & Retail'



In [ ]:
%%bigquery
select r.sub_category, a.icao, a.name as airport, a.city, a.state_prov, c.name as country, r.sentiment, count(*) as count
from air_travel_int.Airport_Review r join air_travel_int.Airport a
on r.airport_icao = a.icao
join air_travel_int.Country c
on a.country_code = c.iso_code
where r.sub_category in ('Wayfinding & Information', 'Airport Staff',
  'Amenities & Facilities', 'Boarding Process', 'Security Screening',
  'Food, Beverage & Retail')
group by r.sub_category, a.icao, a.name, a.city, a.state_prov, c.name, r.sentiment
order by r.sub_category, count(*) desc, a.name

Query is running:   0%|          |

Downloading:   0%|          |

,sub_category,icao,airport,city,state_prov,country,sentiment,count
0,Airport Staff,RPLL,Ninoy Aquino International Airport,Manila,Metro Manila,Philippines,Negative,22
1,Airport Staff,RPLL,Ninoy Aquino International Airport,Manila,Metro Manila,Philippines,Neutral,14
2,Airport Staff,RPVB,Awang Airport,Cotabato,Maguindanao del Sur,Philippines,Negative,5
3,Airport Staff,FAOR,OR Tambo International Airport,Johannesburg,Gauteng,South Africa,Negative,5
4,Airport Staff,KTEB,Teterboro Airport,Teterboro,New Jersey,United States,Positive,5
...,...,...,...,...,...,...,...,...
841,Wayfinding & Information,KHOU,William P Hobby Airport,Houston,Texas,United States,Neutral,1
842,Wayfinding & Information,CYWM,Williams Harbour Airport,Williams Harbour,Newfoundland and Labrador,Canada,Neutral,1
843,Wayfinding & Information,CYWG,Winnipeg / James Armstrong Richardson Internat...,Winnipeg,Manitoba,Canada,Neutral,1
844,Wayfinding & Information,CZUM,Wunnumin Lake Airport,Wunnumin Lake,ON,Canada,Negative,1


##### Materialize the query results

In [ ]:
%%bigquery
create or replace table air_travel_mrt.passenger_sentiment_airports_top_kpis as
  select r.sub_category, a.icao, a.name as airport, a.city, a.state_prov, c.name as country, r.sentiment, count(*) as count
  from air_travel_int.Airport_Review r
  join air_travel_int.Airport a on r.airport_icao = a.icao
  join air_travel_int.Country c on a.country_code = c.iso_code
  where r.sub_category in ('Wayfinding & Information', 'Airport Staff',
    'Amenities & Facilities', 'Boarding Process', 'Security Screening',
    'Food, Beverage & Retail')
  group by r.sub_category, a.icao, a.name, a.city, a.state_prov, c.name, r.sentiment
  order by r.sub_category, count(*) desc, a.name

Query is running:   0%|          |

""


##### Check the mart

In [ ]:
%%bigquery
select *
from air_travel_mrt.passenger_sentiment_airports_top_kpis

Query is running:   0%|          |

Downloading:   0%|          |

,sub_category,icao,airport,city,state_prov,country,sentiment,count
0,Airport Staff,RPLL,Ninoy Aquino International Airport,Manila,Metro Manila,Philippines,Negative,22
1,Airport Staff,RPLL,Ninoy Aquino International Airport,Manila,Metro Manila,Philippines,Neutral,14
2,Airport Staff,RPVB,Awang Airport,Cotabato,Maguindanao del Sur,Philippines,Negative,5
3,Airport Staff,FAOR,OR Tambo International Airport,Johannesburg,Gauteng,South Africa,Negative,5
4,Airport Staff,KTEB,Teterboro Airport,Teterboro,New Jersey,United States,Positive,5
...,...,...,...,...,...,...,...,...
841,Wayfinding & Information,KHOU,William P Hobby Airport,Houston,Texas,United States,Neutral,1
842,Wayfinding & Information,CYWM,Williams Harbour Airport,Williams Harbour,Newfoundland and Labrador,Canada,Neutral,1
843,Wayfinding & Information,CYWG,Winnipeg / James Armstrong Richardson Internat...,Winnipeg,Manitoba,Canada,Neutral,1
844,Wayfinding & Information,CZUM,Wunnumin Lake Airport,Wunnumin Lake,ON,Canada,Negative,1


In [ ]:
%%bigquery
select *
from air_travel_mrt.passenger_sentiment_airports_top_kpis
where country = 'United States'
and sentiment = 'Positive'
and sub_category = 'Food, Beverage & Retail'
order by count desc

Query is running:   0%|          |

Downloading:   0%|          |

,sub_category,icao,airport,city,state_prov,country,sentiment,count
0,"Food, Beverage & Retail",KMFD,Mansfield Lahm Regional Airport,Mansfield,Ohio,United States,Positive,7
1,"Food, Beverage & Retail",KJGG,Jekyll Island Airport,Jekyll Island,Georgia,United States,Positive,3
2,"Food, Beverage & Retail",KJVL,Southern Wisconsin Regional Airport,Janesville,Wisconsin,United States,Positive,2
3,"Food, Beverage & Retail",KLNR,Tri-County Regional Airport,Lone Rock,Pennsylvania,United States,Positive,2
4,"Food, Beverage & Retail",KJXN,Jackson County Reynolds Field,Jackson,Mississippi,United States,Positive,1
5,"Food, Beverage & Retail",KBAF,Westfield-Barnes Regional Airport,Westfield,Massachusetts,United States,Positive,1
6,"Food, Beverage & Retail",KAPA,Centennial Airport,Denver,Colorado,United States,Positive,1
7,"Food, Beverage & Retail",4G7,Chan Gurney Municipal Airport,Yankton,South Dakota,United States,Positive,1
8,"Food, Beverage & Retail",KCXO,Conroe-North Houston Regional Airport,Conroe,Texas,United States,Positive,1
9,"Food, Beverage & Retail",KMIE,Delaware County Johnson Field,Muncie,Indiana,United States,Positive,1


### **User journey 3**: I am a customer experience advisor for LAX. My goal is to improve the quality and variety of dining options available to passengers at LAX terminals. I want to get a pulse on what dining options are popular at different airports and then figure out how to replicate that here.


##### Query Strategy:
*   Lookup the airports which have positive dining reviews (Airport_Reviews)
*   Find out what dining options they offer
*   Compare those options with what LAX offers and see if there are any ones missing  (Airport, Airport_Concessions, Airport_Establishment, and Food_Beverage)

In [ ]:
%%bigquery
select distinct icao, airport
from air_travel_mrt.passenger_sentiment_airports_top_kpis
where country = 'United States'
and sentiment = 'Positive'
and sub_category = 'Food, Beverage & Retail'
order by icao

Query is running:   0%|          |

Downloading:   0%|          |

,icao,airport
0,4G7,Chan Gurney Municipal Airport
1,K4I7,Putnam County Airport
2,K52F,Gray Army Air Field
3,KAPA,Centennial Airport
4,KAPC,Napa County Airport
5,KBAF,Westfield-Barnes Regional Airport
6,KBFD,Bradford Regional Airport
7,KBJC,Rocky Mountain Metropolitan Airport
8,KBRD,Brainerd Lakes Regional Airport
9,KCLL,Easterwood Field


In [ ]:
%%bigquery
select *
from air_travel_int.Airport_Concessions
where airport_icao in
  (select icao
  from air_travel_mrt.passenger_sentiment_airports_top_kpis
  where country = 'United States'
  and sentiment = 'Positive'
  and sub_category = 'Food, Beverage & Retail')

Query is running:   0%|          |

Downloading: |          |

,airport_icao,terminal,business,location,_data_source,_load_time


##### Unfortunately, the dining data does not overlap with the positive review data, so this is a dead end. Need to figure out an alternative strategy with the data that's available.

##### Instead of restricting our search to the US, look at the dining options that received positive reviews outside the US and see what items they offer. Then, look at what LAX offers by comparison

In [ ]:
%%bigquery
select a.icao, a.name as airport, a.country_code, r.review_date, r.review_subject, r.review_details
from air_travel_int.Airport_Review r join air_travel_int.Airport a
on r.airport_icao = a.icao
where sentiment = 'Positive'
and sub_category = 'Food, Beverage & Retail'
order by icao

Query is running:   0%|          |

Downloading:   0%|          |

,icao,airport,country_code,review_date,review_subject,review_details
0,4G7,Chan Gurney Municipal Airport,US,2012-10-22,DJ's 50's and 60's Diner,Exit the back of the FBO and walk down to the ...
1,CYFD,Dolbeau St Felicien Airport,CA,2008-06-13,Good breakfast,A good place for a $100 hamburger or breakfast...
2,CYKZ,Buttonville Municipal Airport,CA,2007-08-09,Druxy's on site,There is a Druxy's (deli chain) in the FBO. Th...
3,CYOS,Owen Sound / Billy Bishop Regional Airport,CA,2016-03-09,On airport restaurant Tilly's Cafe and Bakery ...,Tilley's Cafe and Bakery located in the termin...
4,CYOW,Ottawa Macdonald-Cartier International Airport,CA,2007-07-30,Tim Horton's at north field,"To add to Tony's comment, there's also a Tim H..."
...,...,...,...,...,...,...
66,PTPN,Pohnpei International Airport,FSM,2007-07-29,PNI,A beautiful place. I wish I could have stayed ...
67,RJDT,Tsushima Airport,JP,2024-03-16,Tsushima Yamaneko Airport 対馬やまねこ空港,Very small (but neat) airport which is served ...
68,TTPP,Piarco International Airport,TT,2022-12-09,Always close to my heart,"Good to buy rum, chocolate and jerk seasoning...."
69,VHHH,Hong Kong International Airport,HK,2007-07-29,"A better, but less interesting place",I've been here a number rof times as a passeng...


#### Revised Query Strategy:
*   Use the LLM to extract the restaurant names from the positive reviews
*   Use the LLM to look up what items are on their menus
*   Join the menu items from the positive reviews with the LAX records in Airport_Concessions, Airport_Establishment, Food_Beverage to see items overlap and which ones are different


In [ ]:
%%bigquery
create or replace table air_travel_tmp.airport_terminal_top_dining_establishments as
  with positive_reviews as (
    select r.airport_icao, r.id as review_id, concat(r.review_subject, ': ', r.review_details) as review_content
    from air_travel_int.Airport_Review r
    join air_travel_int.Airport a on r.airport_icao = a.icao
    where r.sentiment = 'Positive'
    and r.sub_category = 'Food, Beverage & Retail'
  ),
  airport_terminal_top_dining as (
     select airport_icao, review_id, review_content,
      AI.GENERATE(
        ('What restaurant or dining establishment is mentioned in this review? ', review_content, '. Note that the review is for airport icao ', airport_icao, '. If the review does not mention the name of a dining establishment, return None.'),
        connection_id => 'us-central1.vertex-ai-connection',
        endpoint => 'gemini-2.5-flash-lite',
        model_params => JSON '{"generation_config":{"thinking_config": {"thinking_budget": 0}}, "tools": [{"googleSearch": {}}]}',
        output_schema => 'mapped_establishment STRING') as llm_response
   from positive_reviews
  )
select airport_icao, review_id, review_content, llm_response.mapped_establishment as mapped_establishment
from airport_terminal_top_dining

Query is running:   0%|          |

""


#### Check results

In [ ]:
%%bigquery
select *
from air_travel_tmp.airport_terminal_top_dining_establishments
where mapped_establishment is not null

Query is running:   0%|          |

Downloading:   0%|          |

,airport_icao,review_id,review_content,mapped_establishment
0,KPAO,15529,Palo Alto Airport: My home airport. Good chil...,Abundant Air Cafe
1,KOCF,21628,Airport Restaurant: Ocala is a great field to ...,Airport Restaurant
2,CYQS,6749,re: Restaurant?: Reply to @bcrosby: As of a y...,Beanery
3,K52F,324,Friendliest Airport Great Cafe: Check out the ...,Blue Hanger Cafe
4,KBJC,3341,Blue Sky Bistro: Stop in to eat at Blue Sky Bi...,Blue Sky Bistro
...,...,...,...,...
63,CYOW,537,Tim Horton's at north field: To add to Tony's ...,Tim Horton's
64,CYQF,373,Tipitina's: Best food and friendliest service ...,Tipitina's
65,KIGM,162,Kingman airport: KIGM was a military training ...,breakfast/lunch cafe
66,RJDT,523015,Tsushima Yamaneko Airport 対馬やまねこ空港: Very small...,cafe


#### Look-up the top menu items offered by the dining establishments

In [ ]:
%%bigquery
create or replace table air_travel_tmp.airport_terminal_top_dining_items as
  with top_dining_items as (
     select airport_icao, review_id, mapped_establishment as establishment,
      AI.GENERATE(
        ('What are the top menu items offered at ', mapped_establishment, ' at airport ', airport_icao, '? Return your answer as a comma-separated list. If you are not able to find the top items, return None'),
        connection_id => 'us-central1.vertex-ai-connection',
        endpoint => 'gemini-2.5-flash-lite',
        model_params => JSON '{"generation_config":{"thinking_config": {"thinking_budget": 1}}, "tools": [{"googleSearch": {}}]}',
        output_schema => 'items STRING') as llm_response
     from air_travel_tmp.airport_terminal_top_dining_establishments
  )
select airport_icao, review_id, establishment, llm_response.items as items
from top_dining_items

Query is running:   0%|          |

""


#### Check results

In [ ]:
%%bigquery
select *
from air_travel_tmp.airport_terminal_top_dining_items
where items != 'None'

Query is running:   0%|          |

Downloading: |          |

,airport_icao,review_id,establishment,items


#### Switch to thinking model, gemini 2.5 pro, since we got zero results with the flash lite model

In [ ]:
%%bigquery
create or replace table air_travel_tmp.airport_terminal_top_dining_items as
  with top_dining_items as (
     select airport_icao, review_id, mapped_establishment as establishment,
      AI.GENERATE(
        ('What are the top menu items offered at restaurant ', mapped_establishment, '? Return your answer as a comma-separated list. If you cannot find a restaurant by that name, return None.'),
        connection_id => 'us-central1.vertex-ai-connection',
        endpoint => 'gemini-2.5-pro',
        model_params => JSON '{"generation_config":{"thinking_config": {"thinking_budget": 1}}, "tools": [{"googleSearch": {}}]}',
        output_schema => 'items STRING') as llm_response
     from air_travel_tmp.airport_terminal_top_dining_establishments
  )
select airport_icao, review_id, establishment, llm_response.items as items
from top_dining_items

Query is running:   0%|          |

""


#### Check results

In [ ]:
%%bigquery
select *
from air_travel_tmp.airport_terminal_top_dining_items
where items != 'None'

Query is running:   0%|          |

Downloading: |          |

,airport_icao,review_id,establishment,items


#### This is another dead end :(

#### Another angle is to extract the broad food categories that are mentioned in their reviews in a positive or neutral light. Then compare those food categories with what's available at LAX

In [ ]:
%%bigquery
create or replace table air_travel_tmp.airport_terminal_mentioned_food_keywords as
  with mentioned_foods as (
     select id as review_id, concat(review_subject, ':', review_details) as review_content,
      AI.GENERATE(
        ('If this review mentions food, what food keywords does it mention?',
          concat(review_subject, ':', review_details), '. Return your answer as a comma-separated list of keywords. If the review does not mention food, return None.'),
        connection_id => 'us-central1.vertex-ai-connection',
        endpoint => 'gemini-2.5-flash-lite',
        model_params => JSON '{"generation_config":{"thinking_config": {"thinking_budget": 0}}, "tools": [{"googleSearch": {}}]}',
        output_schema => 'keywords STRING') as llm_response
     from air_travel_int.Airport_Review
     where sentiment != 'Negative'
  )
select review_id, review_content, llm_response.keywords as keywords
from mentioned_foods

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select *
from air_travel_tmp.airport_terminal_mentioned_food_keywords
where keywords != 'None'

Query is running:   0%|          |

Downloading:   0%|          |

,review_id,review_content,keywords
0,1473,Nice stop:Lake Placid was a wonderful stop. T...,"food, lunch"
1,324,Friendliest Airport Great Cafe:Check out the B...,Cafe
2,15558,Peterborough Airport Restaurant:http://groups....,menu
3,7318,It's a very friendly airport now!:The rude con...,"snack,soda,coffee,sandwiches,food"
4,145,Restaurant on field.:Can't say the Italian dis...,Italian dish
...,...,...,...
218,179,Pick Up Your New Pilatus PC-12:Stopped here en...,"BBQ, burgers"
219,11010,Encouragement !!!:I believe there is nothing t...,"mangoes,sohan halwa"
220,153,My old home base.:I first went up in a single ...,"restaurant, , , Wendy's International"
221,632,Lock Haven:Great airport! FBO will provide gr...,meal


#### Retrieve the food categories we have standardized on

In [ ]:
%%bigquery
select array_to_string(array_agg(distinct sub_category), ', ') as categories
from air_travel_int.Airport_Establishment
where dining = True

#### Map food keywords to category

In [ ]:
%%bigquery
create or replace table air_travel_tmp.airport_terminal_food_keywords_category as
  with canonical_food_categories as (
    select array_to_string(array_agg(distinct sub_category), ', ') as categories
    from air_travel_int.Airport_Establishment
    where dining = True
  ),
  mapped_food_categories as (
     select review_id, review_content, keywords,
      AI.GENERATE(
        ('Map the food keywords , ', keywords, ' to its best fit category: ', categories,
          'If the keywords are too generic, return None.'),
        connection_id => 'us-central1.vertex-ai-connection',
        endpoint => 'gemini-2.5-flash-lite',
        model_params => JSON '{"generation_config":{"thinking_config": {"thinking_budget": 0}}, "tools": [{"googleSearch": {}}]}',
        output_schema => 'category STRING') as llm_response
     from air_travel_tmp.airport_terminal_mentioned_food_keywords
     cross join canonical_food_categories
     where keywords != 'None'
  )
select review_id, review_content, keywords, llm_response.category as category
from mapped_food_categories

Query is running:   0%|          |

""


#### Check results

In [ ]:
%%bigquery
select category, count(*) as num_mentions
from air_travel_tmp.airport_terminal_food_keywords_category
where category != 'None'
group by category
order by category

Query is running:   0%|          |

Downloading:   0%|          |

,category,num_mentions
0,Build-Your-Own Concepts,15
1,Casual Dining Restaurants,81
2,Gourmet Burgers & Sandwiches,33
3,Pizzerias,5
4,Sports Bars,1
5,Themed Restaurants & Sports Bars,2
6,Upscale & Chef-Driven Restaurants,4


#### Compared with availability at LAX

In [ ]:
%%bigquery
select sub_category, count(*) as lax_availability_count
from air_travel_int.Airport_Establishment e
join air_travel_int.Airport_Concessions c
on e.business = c.business
where e.dining = True
and c.airport_icao = 'KLAX'
group by sub_category
order by sub_category

Query is running:   0%|          |

Downloading:   0%|          |

,sub_category,lax_availability_count
0,Build-Your-Own Concepts,1
1,Casual Dining Restaurants,16
2,Gourmet Burgers & Sandwiches,4
3,Themed Restaurants & Sports Bars,3
4,Upscale & Chef-Driven Restaurants,3


#### Merge both views

In [ ]:
%%bigquery
with lax_categories as (
  select sub_category, count(*) as lax_availability_count
  from air_travel_int.Airport_Establishment e
  join air_travel_int.Airport_Concessions c
  on e.business = c.business
  where e.dining = True
  and c.airport_icao = 'KLAX'
  group by sub_category
  order by sub_category
),
reviews_categories as (
  select category, count(*) as num_reviews_mentions
  from air_travel_tmp.airport_terminal_food_keywords_category
  where category != 'None'
  group by category
  order by category
)

select r.category, r.num_reviews_mentions, l.lax_availability_count
from reviews_categories r join lax_categories l
on r.category = l.sub_category
order by r.category

Query is running:   0%|          |

Downloading:   0%|          |

,category,num_reviews_mentions,lax_availability_count
0,Build-Your-Own Concepts,15,1
1,Casual Dining Restaurants,81,16
2,Gourmet Burgers & Sandwiches,33,4
3,Themed Restaurants & Sports Bars,2,3
4,Upscale & Chef-Driven Restaurants,4,3


#### Create the mart

In [ ]:
%%bigquery
create or replace table air_travel_mrt.food_category_comparison as
  with lax_categories as (
    select sub_category, count(*) as lax_availability_count
    from air_travel_int.Airport_Establishment e
    join air_travel_int.Airport_Concessions c
    on e.business = c.business
    where e.dining = True
    and c.airport_icao = 'KLAX'
    group by sub_category
    order by sub_category
),
reviews_categories as (
  select category, count(*) as num_mentions
  from air_travel_tmp.airport_terminal_food_keywords_category
  where category != 'None'
  group by category
  order by category
)

select r.category, r.num_mentions, l.lax_availability_count
from reviews_categories r join lax_categories l
on r.category = l.sub_category
order by r.category

Query is running:   0%|          |

""


#### Check results

In [ ]:
%%bigquery
select *
from air_travel_mrt.food_category_comparison

Query is running:   0%|          |

Downloading:   0%|          |

,category,num_mentions,lax_availability_count
0,Build-Your-Own Concepts,15,1
1,Casual Dining Restaurants,81,16
2,Gourmet Burgers & Sandwiches,33,4
3,Themed Restaurants & Sports Bars,2,3
4,Upscale & Chef-Driven Restaurants,4,3


### **User journey 4**: As a seasoned traveler, I need to identify and filter out consistently unreliable flights, specifically those with a high frequency of non-weather-related delays. This will allow me to optimize my booking choices and minimize future travel disruptions.

#### Get the number of delayed flights due to carrier-related delays (e.g. mechanical, flight crew, etc.)

In [ ]:
%%bigquery
select ap.state_prov, ap.icao, ap.name as airport, al.name as airline,
  count(fd.carrier_delay_min) carrier_delay_count
from air_travel_int.Flight_Delays fd
join air_travel_int.Airline al
on fd.airline_id = al.id
join air_travel_int.Airport ap
on fd.airport_icao = ap.icao
where carrier_delay_min > 0
and weather_delay_min = 0
group by ap.state_prov, ap.icao, ap.name, al.name
order by ap.state_prov, al.name

#### Get the total number of flights

In [ ]:
%%bigquery
select ap.state_prov, ap.icao, ap.name as airport, al.name as airline,
  count(*) total_flights
from air_travel_int.Flight_Delays fd
join air_travel_int.Airline al
on fd.airline_id = al.id
join air_travel_int.Airport ap
on fd.airport_icao = ap.icao
group by ap.state_prov, ap.icao, ap.name, al.name
order by ap.state_prov, al.name

Query is running:   0%|          |

Downloading:   0%|          |

,state_prov,icao,airport,airline,total_flights
0,Alabama,KHSV,Huntsville International Carl T Jones Field,Atlantic Coast Airlines,3
1,Alabama,KBHM,Birmingham-Shuttlesworth International Airport,Atlantic Coast Airlines,13
2,Alabama,KMOB,Mobile Regional Airport,Atlantic Southeast Airlines,206
3,Alabama,K51D,Dothan Regional Airport,Atlantic Southeast Airlines,206
4,Alabama,KHSV,Huntsville International Carl T Jones Field,Atlantic Southeast Airlines,206
...,...,...,...,...,...
1287,Wyoming,KCPR,Casper-Natrona County International Airport,Allegiant Air,1
1288,Wyoming,KJAC,Jackson Hole Airport,Atlantic Southeast Airlines,26
1289,Wyoming,KJAC,Jackson Hole Airport,Frontier Airlines,16
1290,Wyoming,KCPR,Casper-Natrona County International Airport,Trans States Airlines,27


#### Put it all together to get the flight delay rate as a percentage

In [ ]:
%%bigquery
with delayed_flights as (
  select ap.state_prov, ap.icao, ap.name as airport, al.name as airline,
    count(fd.carrier_delay_min) carrier_delay_count
  from air_travel_int.Flight_Delays fd
  join air_travel_int.Airline al
  on fd.airline_id = al.id
  join air_travel_int.Airport ap
  on fd.airport_icao = ap.icao
  where carrier_delay_min > 0
  and weather_delay_min = 0
  group by ap.state_prov, ap.icao, ap.name, al.name
  order by ap.state_prov, al.name
),
all_flights as (
  select ap.state_prov, ap.icao, ap.name as airport, al.name as airline,
    count(*) as total_flights
  from air_travel_int.Flight_Delays fd
  join air_travel_int.Airline al
    on fd.airline_id = al.id
  join air_travel_int.Airport ap
    on fd.airport_icao = ap.icao
  group by ap.state_prov, ap.icao, ap.name, al.name
  order by ap.state_prov, al.name
)

select state_prov, icao, airport, airline, round(carrier_delay_count / total_flights * 100) as delay_rate_percent
from delayed_flights join all_flights
using (state_prov, icao, airport, airline)
order by delay_rate_percent desc

Query is running:   0%|          |

Downloading:   0%|          |

,state_prov,icao,airport,airline,delay_rate_percent
0,Ohio/Pennsylvania,KYNG,Youngstown Warren Regional Airport,Allegiant Air,100.0
1,Florida,KMTH,The Florida Keys Marathon Airport,Atlantic Southeast Airlines,100.0
2,South Dakota,KRAP,Rapid City Regional Airport,Atlantic Southeast Airlines,100.0
3,Arizona,KBIH,Laughlin-Bullhead Intl,Allegiant Air,100.0
4,Colorado,KSGS,City of Colorado Springs Municipal Airport,Atlantic Southeast Airlines,100.0
...,...,...,...,...,...
1235,Texas,KHOU,William P Hobby Airport,Southwest Airlines,1.0
1236,Nevada,KLAS,McCarran International Airport,Virgin America,1.0
1237,Arkansas,KLIT,Bill & Hillary Clinton National Airport/Adams ...,Envoy Air,1.0
1238,California,KLAX,Los Angeles International Airport,Virgin America,1.0


#### Create the mart

In [ ]:
%%bigquery
create or replace table air_travel_mrt.carrier_flight_delays as
  with delayed_flights as (
    select ap.state_prov, ap.icao, ap.name as airport, al.name as airline,
      count(fd.carrier_delay_min) carrier_delay_count
    from air_travel_int.Flight_Delays fd
    join air_travel_int.Airline al
      on fd.airline_id = al.id
    join air_travel_int.Airport ap
      on fd.airport_icao = ap.icao
    where carrier_delay_min > 0
    and weather_delay_min = 0
    group by ap.state_prov, ap.icao, ap.name, al.name
    order by ap.state_prov, al.name
  ),
  all_flights as (
    select ap.state_prov, ap.icao, ap.name as airport, al.name as airline,
      count(*) as total_flights
    from air_travel_int.Flight_Delays fd
    join air_travel_int.Airline al
      on fd.airline_id = al.id
    join air_travel_int.Airport ap
      on fd.airport_icao = ap.icao
    group by ap.state_prov, ap.icao, ap.name, al.name
    order by ap.state_prov, al.name
)

select state_prov, icao, airport, airline, carrier_delay_count, total_flights,
  round(carrier_delay_count / total_flights * 100) as delay_rate_percent
from delayed_flights join all_flights
using (state_prov, icao, airport, airline)
order by delay_rate_percent desc

Query is running:   0%|          |

""


#### Check results

In [ ]:
%%bigquery
select *
from air_travel_mrt.carrier_flight_delays
order by delay_rate_percent desc

Query is running:   0%|          |

Downloading:   0%|          |

,state_prov,icao,airport,airline,carrier_delay_count,total_flights,delay_rate_percent
0,Michigan,KFNT,Bishop International Airport,Atlantic Coast Airlines,2,2,100.0
1,Maryland,KBWI,Baltimore/Washington International Thurgood Ma...,Atlantic Coast Airlines,1,1,100.0
2,Florida,KMTH,The Florida Keys Marathon Airport,Atlantic Southeast Airlines,10,10,100.0
3,South Dakota,KRAP,Rapid City Regional Airport,Atlantic Southeast Airlines,4,4,100.0
4,Colorado,KSGS,City of Colorado Springs Municipal Airport,Atlantic Southeast Airlines,6,6,100.0
...,...,...,...,...,...,...,...
1235,Texas,KAUS,Austin Bergstrom International Airport,Southwest Airlines,1,76,1.0
1236,Tennessee,KBNA,Nashville International Airport,Southwest Airlines,1,76,1.0
1237,Texas,KHOU,William P Hobby Airport,Southwest Airlines,1,76,1.0
1238,Arkansas,KLIT,Bill & Hillary Clinton National Airport/Adams ...,Envoy Air,1,97,1.0


### **User journey 5**: I'm a transportation researcher and my focus is on predictive maintenance. I'd like to understand which aircraft models are experiencing the greatest maintenance-related delays (relative to their total flights).  

#### Get the flight delay counts by aircraft

In [9]:
%%bigquery
select ac.name as aircraft, ac.icao as aircraft_code, count(*) aircraft_delay_count
    from air_travel_int.Flight_Delays fd
    join air_travel_int.Airline al
      on fd.airline_id = al.id
    join air_travel_int.Airport ap
      on fd.airport_icao = ap.icao
    join air_travel_int.Route_Equipment re
      on al.id = re.airline_id and ap.icao = re.dest_airport_icao
    join air_travel_int.Aircraft ac
      on re.icao_equipment = ac.icao
    where carrier_delay_min > 0
    and weather_delay_min = 0
    group by ac.name, ac.icao
    order by ac.name

Query is running:   0%|          |

Downloading:   0%|          |

,aircraft,aircraft_code,aircraft_delay_count
0,Airbus A319,A319,4465
1,Airbus A320,A320,11650
2,Airbus A321,A321,38
3,Airbus A340-600,A346,1
4,Airbus A380,A388,40
5,Airbus A380-800,A388,40
6,Beechcraft 1900,B190,1
7,Boeing 717,B712,283
8,Boeing 737,B737,45
9,Boeing 737-300,B733,2339


#### Get the flight counts by aircraft (for all flights, on-time and delayed)

In [13]:
%%bigquery
select ac.name as aircraft, ac.icao as aircraft_code, count(*) aircraft_flight_count
from air_travel_int.Flight_Delays fd
join air_travel_int.Airline al
  on fd.airline_id = al.id
join air_travel_int.Airport ap
  on fd.airport_icao = ap.icao
join air_travel_int.Route_Equipment re
  on al.id = re.airline_id and ap.icao = re.dest_airport_icao
join air_travel_int.Aircraft ac
  on re.icao_equipment = ac.icao
group by ac.name, ac.icao
order by ac.name

Query is running:   0%|          |

Downloading:   0%|          |

,aircraft,aircraft_code,aircraft_flight_count
0,Airbus A319,A319,22552
1,Airbus A320,A320,66752
2,Airbus A321,A321,1437
3,Airbus A340-600,A346,251
4,Airbus A380,A388,438
5,Airbus A380-800,A388,438
6,Beechcraft 1900,B190,76
7,Boeing 717,B712,2432
8,Boeing 737,B737,684
9,Boeing 737-300,B733,22455


#### Get the aircraft delay rate

In [25]:
%%bigquery
  with delayed_flights as (
    select ac.name as aircraft, ac.icao as aircraft_code, count(*) aircraft_delay_flights
    from air_travel_int.Flight_Delays fd
    join air_travel_int.Airline al
      on fd.airline_id = al.id
    join air_travel_int.Airport ap
      on fd.airport_icao = ap.icao
    join air_travel_int.Route_Equipment re
      on al.id = re.airline_id and ap.icao = re.dest_airport_icao
    join air_travel_int.Aircraft ac
      on re.icao_equipment = ac.icao
    where carrier_delay_min > 0
    and weather_delay_min = 0
    group by ac.name, ac.icao
    order by ac.name
  ),
  total_flights as (
    select ac.name as aircraft, ac.icao as aircraft_code, count(*) aircraft_total_flights
    from air_travel_int.Flight_Delays fd
    join air_travel_int.Airline al
    on fd.airline_id = al.id
    join air_travel_int.Airport ap
    on fd.airport_icao = ap.icao
    join air_travel_int.Route_Equipment re
    on al.id = re.airline_id and ap.icao = re.dest_airport_icao
    join air_travel_int.Aircraft ac
    on re.icao_equipment = ac.icao
    group by ac.name, ac.icao
    order by ac.name
  )

select aircraft, aircraft_code, aircraft_delay_flights, aircraft_total_flights, round(aircraft_delay_flights / aircraft_total_flights * 100) as aircraft_delay_rate
from delayed_flights
join total_flights
using (aircraft, aircraft_code)
order by aircraft_delay_rate desc


Query is running:   0%|          |

Downloading:   0%|          |

,aircraft,aircraft_code,aircraft_delay_flights,aircraft_total_flights,aircraft_delay_rate
0,Boeing 777-300ER,B77W,265,948,28.0
1,Boeing 777-200LR,B77L,106,387,27.0
2,Boeing 757,B757,134,607,22.0
3,Embraer 190,E190,5453,27897,20.0
4,Airbus A319,A319,4465,22552,20.0
5,Airbus A320,A320,11650,66752,17.0
6,Boeing 737-500,B735,714,5203,14.0
7,Boeing 717,B712,283,2432,12.0
8,Boeing 737-300,B733,2339,22455,10.0
9,Airbus A380,A388,40,438,9.0


#### Create the mart

In [24]:
%%bigquery
create or replace table air_travel_mrt.aircraft_flight_delays as
  with delayed_flights as (
    select ac.name as aircraft, ac.icao as aircraft_code, count(*) aircraft_delay_flights
    from air_travel_int.Flight_Delays fd
    join air_travel_int.Airline al
      on fd.airline_id = al.id
    join air_travel_int.Airport ap
      on fd.airport_icao = ap.icao
    join air_travel_int.Route_Equipment re
      on al.id = re.airline_id and ap.icao = re.dest_airport_icao
    join air_travel_int.Aircraft ac
      on re.icao_equipment = ac.icao
    where carrier_delay_min > 0
    and weather_delay_min = 0
    group by ac.name, ac.icao
    order by ac.name
  ),
  total_flights as (
    select ac.name as aircraft, ac.icao as aircraft_code, count(*) aircraft_total_flights
    from air_travel_int.Flight_Delays fd
    join air_travel_int.Airline al
    on fd.airline_id = al.id
    join air_travel_int.Airport ap
    on fd.airport_icao = ap.icao
    join air_travel_int.Route_Equipment re
    on al.id = re.airline_id and ap.icao = re.dest_airport_icao
    join air_travel_int.Aircraft ac
    on re.icao_equipment = ac.icao
    group by ac.name, ac.icao
    order by ac.name
  )

select aircraft, aircraft_code, aircraft_delay_flights, aircraft_total_flights, round(aircraft_delay_flights / aircraft_total_flights * 100) as aircraft_delay_rate
from delayed_flights
join total_flights
using (aircraft, aircraft_code)
order by aircraft_delay_rate desc

Query is running:   0%|          |

""


#### Check results

In [26]:
%%bigquery
select *
from air_travel_mrt.aircraft_flight_delays
order by aircraft_delay_rate desc

Query is running:   0%|          |

Downloading:   0%|          |

,aircraft,aircraft_code,aircraft_delay_flights,aircraft_total_flights,aircraft_delay_rate
0,Boeing 777-300ER,B77W,265,948,28.0
1,Boeing 777-200LR,B77L,106,387,27.0
2,Boeing 757,B757,134,607,22.0
3,Airbus A319,A319,4465,22552,20.0
4,Embraer 190,E190,5453,27897,20.0
5,Airbus A320,A320,11650,66752,17.0
6,Boeing 737-500,B735,714,5203,14.0
7,Boeing 717,B712,283,2432,12.0
8,Boeing 737-300,B733,2339,22455,10.0
9,Airbus A380,A388,40,438,9.0


#### Future work: Add the FAA Service Difficulty Reports (SDR) data to the lakehouse. This would allow us to cross-reference tail numbers and see if a delay coincided with a reported part failure.